Google Big Query is a distributed data warehouse built on a serverless architecture . We’ll discuss this framework in class. In this task you’ll upload all Wedge transaction records to Google Big Query. You’ll want to make sure that the column data types are correctly specified and you’ve properly handled the null values. 
The requirements for this task change depending on the grade you’re going for. 
Note: this assignment can be done manually or programmatically. Naturally I’d prefer it be done programmatically so that you get more practice, but that’s not required to get full credit.

1. Clean the data
    a. I need to split on the delimiter
    b. check for a header (and add it if it doesn't have one)
    c. fix the \\N and \N  and NULL values - keep as NULL
    d. Split them into single month dataframes
2. Upload to GBQ
    a. Upload each one as a separate table in a new dataset in my GBQ project


In [1]:
import os
import pandas as pd
import zipfile
import csv
import io

from zipfile import ZipFile
from io import TextIOWrapper
from shutil import move

In [6]:
zip_files = os.listdir("WedgeZipOfZips")

In [ ]:
delimiters = dict()

# Start by reading in all the files again.
for this_zf in zip_files:
    with ZipFile("WedgeZipOfZips/" + this_zf, 'r') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file = zf.open(file_name, 'r')
            input_file = io.TextIOWrapper(input_file, encoding="utf-8")

            # Read the first line to detect the delimiter
            first_line = input_file.readline()
            dialect = csv.Sniffer().sniff(sample=first_line, delimiters=[",", ";", "\t"])
            detected_delimiter = dialect.delimiter

            # Check if the detected delimiter is different from ","
            if detected_delimiter != ",":
                # Change the delimiter to ","
                delimiters[file_name] = ","
            else:
                delimiters[file_name] = detected_delimiter

            # Reset the file back to the beginning for further processing
            input_file.seek(0)

            # Now, you can process the file using the appropriate delimiter
            for line in input_file:
                # Process the data rows here
                data = line.strip().split(delimiters[file_name])
                

            print(f"{file_name}: has delimiter: {detected_delimiter}")
            input_file.close()  # tidy up

            if detected_delimiter == ";":
                move(file_name, os.path.join(semicolon_folder, file_name))

In [ ]:
headers = dict()

for this_zf in zip_files:
    with ZipFile("WedgeZipOfZips/" + this_zf, 'r') as zf:
        zipped_files = zf.namelist()

        for file_name in zipped_files:
            input_file = zf.open(file_name, 'r')
            input_file = io.TextIOWrapper(input_file, encoding="utf-8")

            this_delimiter = delimiters[file_name]

            # Read the first line to check for the header row
            first_line = input_file.readline()

            # Check if the first line is a header row (you can customize this check)
            is_header = any(keyword in first_line for keyword in ['datetime', 'register_no', 'description', 'trans_status', 'quantity'])

            headers[file_name] = is_header

            print(f"File: {file_name}, Has Header: {is_header}")

            input_file.close()  # tidy up

            # Move the file to our no_headers folder
            if is_header:
                move(file_name, os.path.join(no_headers, file_name))